In [2]:
import pandas as pd
import ee
import geemap
import imageio
from matplotlib import pyplot as plt
import sys
import warnings
import cv2
warnings.filterwarnings('ignore')





## Create a conda environment

Use the following command to create the conda environment:

```bash
conda create --name <env> --file requirements.txt
```




In [3]:
# import local modules
sys.path.append('../src')
from Fishnet import Fishnet
from ImageProcessor import ImageProcessor

# update files automatically
%load_ext autoreload
%autoreload 2

# Authenticate to Earth Engine
ee.Initialize(project="jpmorgancapstone")

# Fishnet

First, we can generate the fishnet using the following code. 

- We instantiate a `Fishnet` object 
- We generate the fishnet
- We aggregated the fishnet into batches corresponding to the images to extract froom the GoogleEarth Engine

In [ ]:
# Create an instance of the Fishnet class
fishnet_creator_example = Fishnet('../Gis/Texas_State_Boundary/State.shp', 0.25, overlay_method='union', clip=False) 

# Create the fishnet
fishnet_creator_example.create_fishnet()

# creates batches of 4miles x 4miles
fishnet_creator_example.batch(4)  

In [ ]:
fishnet_creator_example.plot_fishnet()

## Filtering

We can filter the `Fishnet` to one specific region to avoid working on the whole area, simply to increase computational speed.

In [ ]:
region = [-95.799944,29.374853,-95.028636,29.795492]

# Filter the fishnet by the larger bounding box
fishnet_creator_example.filter_fishnet_by_bbox(region)
fishnet_creator_example.filtered_fishnet.head(10)

## Save and load the fishnet object

In [ ]:
# this specific fishnet was 0.25miles granular and 4miles edge of each batch.
fishnet_creator_example.save('../Gis/Fishnet/fishnet_quarter_mile.pkl') 

# load the object
fishnet_creator_example = Fishnet.load('../Gis/Fishnet/fishnet_quarter_mile.pkl')

In [ ]:
# 